In [1]:
import os
import torch
import math
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,io,datasets
import torch.utils.data as data_utils
from torch.optim import SGD,Adam,lr_scheduler
import torchvision.models as models

In [2]:
rootDir="group_3/"
trainDir="train"
valDir="valididation"
testDir="test"
batchSize=128
device=torch.device("cuda:2")
p=0.1

In [3]:
"""class CustomTensorDataset(Dataset):
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)"""

'class CustomTensorDataset(Dataset):\n    def __init__(self, tensors, transform=None):\n        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)\n        self.tensors = tensors\n        self.transform = transform\n\n    def __getitem__(self, index):\n        x = self.tensors[0][index]\n\n        if self.transform:\n            x = self.transform(x)\n\n        y = self.tensors[1][index]\n\n        return x, y\n\n    def __len__(self):\n        return self.tensors[0].size(0)'

In [4]:
def loadData(dataset,typeData="train"):
    X=[]
    Y=[]
    data=dataset[typeData]["image"]
    label=dataset[typeData]["label"]
    for i in range(len(data)):
        img=np.array(data[i])
        if img.ndim==3:
            X.append(img)
            Y.append(label[i])
    return torch.tensor(np.array(X)),torch.tensor(np.array(Y).reshape(-1,1))

In [5]:
class Classifier(torch.nn.Module):
    def __init__(self, n_in, n_out,modelVGGFeatureMap,avgPool ,layers,activationFn=torch.nn.ReLU() ,dropout=False,BN=False):
        super(Classifier, self).__init__()
        self.modelVGGFeatureMap=modelVGGFeatureMap
        self.avgPool=avgPool        
        net=[]
        for i in range(len(layers)-2):
            net.append(torch.nn.Linear(layers[i],layers[i+1]))
            if BN and i==0:
                net.append(torch.nn.BatchNorm1d(layers[i+1]))
            net.append(activationFn)
            if dropout and i==0:
                net.append(torch.nn.Dropout(p))

        net.append(torch.nn.Linear(layers[-2],layers[-1]))
        self.fc = torch.nn.Sequential(*net)
        

    def forward(self, X):
        features=self.modelVGGFeatureMap(X)
        features=self.avgPool(features)
        features=features.reshape(features.shape[0],-1)
        
        output=self.fc(features)
        return output

In [6]:
def trainModel(model,epochs=30, returnLoss=False, returnAccuracy=False , saveModelName="bestModel"):
    epoch_step_size=1
    bestValidAccuracy=-1
    
    lossList=[]
    trainaccuracyList=[]
    validaccuracyList=[]
    
    print("Started training the network")
    startTime=time.time()
    for epoch in range(epochs):
        print()
        print("Starting epoch no ",str(epoch))
        model.train()
        runningLoss=0
        for batch_idx, (trainData, trainTarget) in enumerate(trainLoader):
            #l1Norm=0
            optimizer.zero_grad()
            trainData, trainTarget = trainData.to(device), trainTarget.reshape(-1).type(torch.LongTensor).to(device)
            output = model(trainData)
            loss = lossFn(output, trainTarget)
            #if epoch>6:
            #     l1Norm = 0.01*sum(torch.linalg.norm(p, 2) for p in model.fc.parameters())
            
            loss=loss#+l1Norm
            loss.backward()
            optimizer.step()    
            runningLoss+=loss.item()
        
        trainAccuracy=findAccuracy(model,trainLoader)
        validAccuracy=findAccuracy(model,validLoader)
        
        if returnLoss:
            lossList.append(runningLoss)
        
        if returnAccuracy:
            trainaccuracyList.append(trainAccuracy)
            validaccuracyList.append(validAccuracy)
            
        if validAccuracy>bestValidAccuracy:
            print("Saving the model")
            torch.save(model.state_dict(), saveModelName+'.pth')
            bestValidAccuracy=validAccuracy
            
        scheduler.step()
        if epoch%epoch_step_size==0:
            print("Loss: ",str(runningLoss)+" Train Accuracy is ",str(trainAccuracy)+" Valid Accuracy is ",str(validAccuracy))
    
    model.load_state_dict(torch.load(saveModelName+".pth"))
    endTime=time.time()
    print("Training finished in ",str(endTime-startTime)+" seconds")
    return model,lossList, trainaccuracyList, validaccuracyList

In [7]:
def findAccuracy(model,loader):
    model.eval()
    accuracy = 0.0
    total = 0.0
  
    with torch.no_grad():
        for batch_idx, (images,labels) in enumerate(loader):
            images,labels = images.to(device), labels.to(device)
            outputs = model(images)
            Ypred = torch.argmax(outputs, 1)
            total += images.size(0)
            accuracy += (Ypred.reshape(-1) == labels.reshape(-1)).sum().item()
    accuracy = (100 * accuracy / total)
    return accuracy

In [8]:
#dataset = load_dataset("imagefolder", data_dir=rootDir)

In [9]:
"""trainPreprocess = transform =   transforms.Compose([
        #transforms.Resize((224, 224)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.RandomAffine(degrees=15, translate=None, scale=(1, 2), shear=5, resample=False, fillcolor=0),
        
])


testPreprocess = transform =   transforms.Compose([
        #transforms.Resize((224, 224)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))        
])"""

'trainPreprocess = transform =   transforms.Compose([\n        #transforms.Resize((224, 224)),\n        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),\n        transforms.RandomHorizontalFlip(),\n        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),\n        transforms.RandomAffine(degrees=15, translate=None, scale=(1, 2), shear=5, resample=False, fillcolor=0),\n        \n])\n\n\ntestPreprocess = transform =   transforms.Compose([\n        #transforms.Resize((224, 224)),\n        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))        \n])'

In [10]:
"""tensorXtrain,tensorYtrain=loadData(dataset)
tensorXtest,tensorYtest=loadData(dataset,typeData="test")
tensorXvalid,tensorYvalid=loadData(dataset,typeData="validation")"""

'tensorXtrain,tensorYtrain=loadData(dataset)\ntensorXtest,tensorYtest=loadData(dataset,typeData="test")\ntensorXvalid,tensorYvalid=loadData(dataset,typeData="validation")'

In [11]:
"""tensorXtrain=tensorXtrain.reshape(tensorXtrain.shape[0],3,224,224)/255
tensorXtest=tensorXtest.reshape(tensorXtest.shape[0],3,224,224)/255
tensorXvalid=tensorXvalid.reshape(tensorXvalid.shape[0],3,224,224)/255
"""

'tensorXtrain=tensorXtrain.reshape(tensorXtrain.shape[0],3,224,224)/255\ntensorXtest=tensorXtest.reshape(tensorXtest.shape[0],3,224,224)/255\ntensorXvalid=tensorXvalid.reshape(tensorXvalid.shape[0],3,224,224)/255\n'

In [12]:
"""trainDataset = CustomTensorDataset(tensors=(tensorXtrain, tensorYtrain), transform=testPreprocess)
trainLoader = data_utils.DataLoader(trainDataset, batch_size=batchSize , shuffle=True)

testDataset = CustomTensorDataset(tensors=(tensorXtest, tensorYtest), transform=testPreprocess)
testLoader = data_utils.DataLoader(testDataset, batch_size=batchSize , shuffle=True)

validDataset = CustomTensorDataset(tensors=(tensorXvalid, tensorYvalid), transform=testPreprocess)
validLoader = data_utils.DataLoader(validDataset, batch_size=batchSize , shuffle=True)"""

'trainDataset = CustomTensorDataset(tensors=(tensorXtrain, tensorYtrain), transform=testPreprocess)\ntrainLoader = data_utils.DataLoader(trainDataset, batch_size=batchSize , shuffle=True)\n\ntestDataset = CustomTensorDataset(tensors=(tensorXtest, tensorYtest), transform=testPreprocess)\ntestLoader = data_utils.DataLoader(testDataset, batch_size=batchSize , shuffle=True)\n\nvalidDataset = CustomTensorDataset(tensors=(tensorXvalid, tensorYvalid), transform=testPreprocess)\nvalidLoader = data_utils.DataLoader(validDataset, batch_size=batchSize , shuffle=True)'

In [13]:
rootDir = "group_3"
trainPath = rootDir+'/train'
valPath = rootDir+'/valid'
testPath = rootDir+'/test'

In [14]:
# Define transforms for image pre-processing

trainTransform =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

In [15]:
trainSet = datasets.ImageFolder(trainPath, transform=transform)
valSet = datasets.ImageFolder(valPath, transform=transform)
testSet = datasets.ImageFolder(testPath, transform=transform)

trainLoader = DataLoader(trainSet, batch_size = batchSize, shuffle=True)
validLoader = DataLoader(valSet, batch_size = batchSize)
testLoader = DataLoader(testSet, batch_size = batchSize)

In [16]:
#extracting the VGG pretrained network
modelVGG = models.vgg16(pretrained=True)
modelVGGFeatureMap=modelVGG.features.to(device)
avgPool=modelVGG.avgpool.to(device)

In [17]:
#No of output classes
#learning rate for the optimizer
#should we include dropout in fully connected layers
#layer architecture for fully connected layers
numClass=len(os.listdir(trainPath))
lr=0.01
dropout=True
BN=False
activationFn=torch.nn.ReLU()
layers=[25088,4096,4096,25]
saveModelName="bestModelDrop"

In [18]:
torch.manual_seed(42)
model=Classifier(25088, numClass,modelVGGFeatureMap,avgPool,layers,activationFn=activationFn,dropout=dropout,BN=BN).to(device).float()
optimizer = SGD(model.parameters(), lr=lr,momentum=0.8, weight_decay=5e-4)
#optimizer=Adam(model.parameters(),lr=lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,10,15], gamma=0.1)
lossFn=torch.nn.CrossEntropyLoss()

In [19]:
for param in model.modelVGGFeatureMap.parameters():
    param.requires_grad = False

In [20]:
model

Classifier(
  (modelVGGFeatureMap): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padd

In [21]:
print("The accuracy before training the model on training data was ",findAccuracy(model,trainLoader))
print("The accuracy before training the model on testing data was ",findAccuracy(model,testLoader))

The accuracy before training the model on training data was  4.925241864555849
The accuracy before training the model on testing data was  2.4


In [22]:
epochs=25
model,lossList, trainaccuracyList, validaccuracyList=trainModel(model,epochs,returnLoss=True, returnAccuracy=True,saveModelName=saveModelName)

Started training the network

Starting epoch no  0
Saving the model
Loss:  52.089589297771454 Train Accuracy is  90.12019935502785 Valid Accuracy is  88.0

Starting epoch no  1
Saving the model
Loss:  7.327360168099403 Train Accuracy is  97.59601289944298 Valid Accuracy is  92.8

Starting epoch no  2
Saving the model
Loss:  2.092576876282692 Train Accuracy is  99.73614775725594 Valid Accuracy is  99.2

Starting epoch no  3
Loss:  1.0294190980494022 Train Accuracy is  99.97068308413955 Valid Accuracy is  99.2

Starting epoch no  4
Loss:  0.5912543199956417 Train Accuracy is  99.97068308413955 Valid Accuracy is  99.2

Starting epoch no  5
Loss:  0.42855485063046217 Train Accuracy is  99.97068308413955 Valid Accuracy is  99.2

Starting epoch no  6
Loss:  0.40664856880903244 Train Accuracy is  99.97068308413955 Valid Accuracy is  99.2

Starting epoch no  7
Loss:  0.38539939373731613 Train Accuracy is  99.97068308413955 Valid Accuracy is  99.2

Starting epoch no  8
Loss:  0.3747060410678386

In [24]:
print("The accuracy after training the model on testing data is ",findAccuracy(model,testLoader))
print("The accuracy after training the model on validation data is ",findAccuracy(model, validLoader))

The accuracy after training the model on testing data is  98.4
The accuracy after training the model on validation data is  99.2


In [ ]:
plt.plot(lossList)
plt.xlabel("epoch number")
plt.ylabel("Total Loss")

In [ ]:
plt.plot(trainaccuracyList)
plt.plot(validaccuracyList)
plt.xlabel("epoch number")
plt.ylabel("Accuracy")
plt.legend(["training accuracy", "validation accuracy"], loc ="lower right")